In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
os.chdir("../")
from datasets import load_dataset, Dataset, DatasetDict
from src.data import HF_Dataset
from src.conf import HF_Dataset_Params
from src.utils import get_split_limit

In [3]:
dataset = load_dataset("monash_tsf", "tourism_monthly")
freq = "1H"
prediction_length = 48

Generating train split:   0%|          | 0/366 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/366 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/366 [00:00<?, ? examples/s]

Dataset monash_tsf downloaded and prepared to /home/onyxia/.cache/huggingface/datasets/monash_tsf/tourism_monthly/1.0.0/82998723d55d6edbc664cfaebe371004d849846ee5f61b57f1f974000d44c050. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset["train"]

Dataset({
    features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
    num_rows: 366
})

In [10]:
params = HF_Dataset_Params(
    [datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 12, 0, 0)], 
    [[i for i in range(100)], [-i for i in range(100)]],
    [[], []],
    [[], []],
    np.array([0.7, 0.2, 0.1]),
    "1H"
)
ds_obj = HF_Dataset(params)

In [14]:
ds_obj.get_num_of_variates()

2

In [11]:
multi_variate_train_example, multi_variate_test_example = ds_obj.multi_variate_datasets()

In [12]:
multi_variate_train_example

{'target': array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
          11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
          22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
          33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
          44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
          55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
          66.,  67.,  68.,  69.],
        [  0.,  -1.,  -2.,  -3.,  -4.,  -5.,  -6.,  -7.,  -8.,  -9., -10.,
         -11., -12., -13., -14., -15., -16., -17., -18., -19., -20., -21.,
         -22., -23., -24., -25., -26., -27., -28., -29., -30., -31., -32.,
         -33., -34., -35., -36., -37., -38., -39., -40., -41., -42., -43.,
         -44., -45., -46., -47., -48., -49., -50., -51., -52., -53., -54.,
         -55., -56., -57., -58., -59., -60., -61., -62., -63., -64., -65.,
         -66., -67., -68., -69.]], dtype=float32),
 'fea

In [13]:
multi_variate_test_example

{'target': array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
          11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
          22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
          33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
          44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
          55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
          66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
          77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
          88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
          99.],
        [  0.,  -1.,  -2.,  -3.,  -4.,  -5.,  -6.,  -7.,  -8.,  -9., -10.,
         -11., -12., -13., -14., -15., -16., -17., -18., -19., -20., -21.,
         -22., -23., -24., -25., -26., -27., -28., -29., -30., -31., -32.,
         -33., -34., -35., -36., -37., -38., -39., -40., -41., -42., -43.,

In [9]:
from transformers import InformerConfig, InformerForPrediction

config = InformerConfig(
    # in the multivariate setting, input_size is the number of variates in the time series per time step
    input_size=num_of_variates,
    # prediction length:
    prediction_length=prediction_length,
    # context length:
    context_length=prediction_length * 2,
    # lags value copied from 1 week before:
    lags_sequence=[1, 24 * 7],
    # we'll add 5 time features ("hour_of_day", ..., and "age"):
    num_time_features=len(time_features) + 1,
    # informer params:
    dropout=0.1,
    encoder_layers=6,
    decoder_layers=4,
    # project input from num_of_variates*len(lags_sequence)+num_time_features to:
    d_model=64,
)

model = InformerForPrediction(config)

NameError: name 'num_of_variates' is not defined